In [2]:
# 自动计算cell的计算时间
%load_ext autotime

%matplotlib inline
%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

time: 523 ms (started: 2021-08-26 22:53:22 +08:00)


In [ ]:
%%bash

# 增加更新
git add *.ipynb *.md

git remote -v

git commit -m '更新 #9  Aug 26, 2021'

#git push origin master
git push

In [ ]:
%%bash

# 增加更新
git add *.ipynb *.md

git remote -v

git commit -m '更新 #1 Aug 26, 2021'

#git push origin master
git push

In [3]:
#设置使用的gpu
import tensorflow as tf
from tensorflow import keras

gpus = tf.config.list_physical_devices("GPU")

if gpus:
   
    gpu0 = gpus[0] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True) #设置GPU显存用量按需使用
    # 或者也可以设置GPU显存为固定使用量(例如：4G)
    #tf.config.experimental.set_virtual_device_configuration(gpu0,
    #    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) 
    tf.config.set_visible_devices([gpu0],"GPU")

time: 3.09 s (started: 2021-08-26 22:54:14 +08:00)


In [4]:
tf.__version__

'2.6.0'

time: 5.88 ms (started: 2021-08-26 22:54:40 +08:00)


# 简单的自定义层示例：Antirectifier

## 介绍
此示例展示了如何使用 Antirectifier 层（最初在 2016 年 1 月作为 Keras 示例脚本提出）创建自定义层，这是 ReLU 的替代方案。 它不是将输入的负部分归零，而是将负部分和正部分分开，并返回两者绝对值的串联。 这避免了信息丢失，代价是维度增加。 为了修复维数增加，我们将特征线性组合回原始大小的空间。

## 设置

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

time: 1.1 s (started: 2021-08-26 22:58:13 +08:00)


## 抗整流层

In [6]:
class Antirectifier(layers.Layer):
    def __init__(self, initializer="he_normal", **kwargs):
        super(Antirectifier, self).__init__(**kwargs)
        self.initializer = keras.initializers.get(initializer)

    def build(self, input_shape):
        output_dim = input_shape[-1]
        self.kernel = self.add_weight(
            shape=(output_dim * 2, output_dim),
            initializer=self.initializer,
            name="kernel",
            trainable=True,
        )

    def call(self, inputs):
        inputs -= tf.reduce_mean(inputs, axis=-1, keepdims=True)
        pos = tf.nn.relu(inputs)
        neg = tf.nn.relu(-inputs)
        concatenated = tf.concat([pos, neg], axis=-1)
        mixed = tf.matmul(concatenated, self.kernel)
        return mixed

    def get_config(self):
        # Implement get_config to enable serialization. This is optional.
        base_config = super(Antirectifier, self).get_config()
        config = {"initializer": keras.initializers.serialize(self.initializer)}
        return dict(list(base_config.items()) + list(config.items()))

time: 2.42 ms (started: 2021-08-26 23:00:43 +08:00)


让我们在 MNIST 上试驾一下

In [7]:
# Training parameters
batch_size = 128
num_classes = 10
epochs = 20

# The data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train /= 255
x_test /= 255
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# Build the model
model = keras.Sequential(
    [
        keras.Input(shape=(784,)),
        layers.Dense(256),
        Antirectifier(),
        layers.Dense(256),
        Antirectifier(),
        layers.Dropout(0.5),
        layers.Dense(10),
    ]
)

# Compile the model
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train the model
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.15)

# Test the model
model.evaluate(x_test, y_test)

60000 train samples
10000 test samples
Epoch 1/20
399/399 [==============================] - 6s 10ms/step - loss: 0.3845 - sparse_categorical_accuracy: 0.8872 - val_loss: 0.2007 - val_sparse_categorical_accuracy: 0.9458
Epoch 2/20
399/399 [==============================] - 4s 10ms/step - loss: 0.1762 - sparse_categorical_accuracy: 0.9511 - val_loss: 0.2024 - val_sparse_categorical_accuracy: 0.9469
Epoch 3/20
399/399 [==============================] - 4s 9ms/step - loss: 0.1402 - sparse_categorical_accuracy: 0.9623 - val_loss: 0.2007 - val_sparse_categorical_accuracy: 0.9542
Epoch 4/20
399/399 [==============================] - 4s 10ms/step - loss: 0.1160 - sparse_categorical_accuracy: 0.9685 - val_loss: 0.3924 - val_sparse_categorical_accuracy: 0.9252
Epoch 5/20
399/399 [==============================] - 4s 9ms/step - loss: 0.1038 - sparse_categorical_accuracy: 0.9733 - val_loss: 0.2550 - val_sparse_categorical_accuracy: 0.9466
Epoch 6/20
399/399 [==============================] - 4s 9

[0.21115773916244507, 0.9761000275611877]

time: 1min 23s (started: 2021-08-26 23:01:14 +08:00)


# 概率贝叶斯神经网络

## 介绍
对深度学习采用概率方法可以考虑不确定性，因此模型可以为不正确的预测分配较少的置信度。 由于标签或模型中的测量误差或噪声，由于模型有效学习的数据可用性不足，可以在数据中找到不确定性的来源。

这个例子演示了如何构建基本的概率贝叶斯神经网络来解释这两种类型的不确定性。 我们使用与 Keras API 兼容的 TensorFlow Probability 库。

此示例需要 TensorFlow 2.3 或更高版本。 您可以使用以下命令安装 Tensorflow Probability：

In [ ]:
pip install tensorflow-probability -U

## 数据集
我们使用 TensorFlow 数据集中提供的 Wine Quality 数据集。 我们使用 red wine 子集，其中包含 4,898 个示例。 该数据集具有葡萄酒的 11 个数值理化特征，任务是预测葡萄酒质量，即 0 到 10 之间的分数。在本示例中，我们将其视为回归任务。

您可以使用以下命令安装 TensorFlow 数据集：

In [ ]:
pip install tensorflow-datasets

## 设置

In [10]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow_probability as tfp

time: 1.47 s (started: 2021-08-27 00:40:15 +08:00)


## 创建训练和评估数据集
在这里，我们使用 tfds.load() 加载 wine_quality 数据集，并将目标特征转换为浮点。 然后，我们打乱数据集并将其拆分为训练集和测试集。 我们将第一个 train_size 示例作为训练分割，其余作为测试分割。

In [11]:
def get_train_and_test_splits(train_size, batch_size=1):
    # We prefetch with a buffer the same size as the dataset because th dataset
    # is very small and fits into memory.
    dataset = (
        tfds.load(name="wine_quality", as_supervised=True, split="train")
        .map(lambda x, y: (x, tf.cast(y, tf.float32)))
        .prefetch(buffer_size=dataset_size)
        .cache()
    )
    # We shuffle with a buffer the same size as the dataset.
    train_dataset = (
        dataset.take(train_size).shuffle(buffer_size=train_size).batch(batch_size)
    )
    test_dataset = dataset.skip(train_size).batch(batch_size)

    return train_dataset, test_dataset

time: 1.13 ms (started: 2021-08-27 00:41:57 +08:00)


## 编译、训练和评估模型

In [12]:
hidden_units = [8, 8]
learning_rate = 0.001


def run_experiment(model, loss, train_dataset, test_dataset):

    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
        loss=loss,
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    print("Start training the model...")
    model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)
    print("Model training finished.")
    _, rmse = model.evaluate(train_dataset, verbose=0)
    print(f"Train RMSE: {round(rmse, 3)}")

    print("Evaluating model performance...")
    _, rmse = model.evaluate(test_dataset, verbose=0)
    print(f"Test RMSE: {round(rmse, 3)}")

time: 1.65 ms (started: 2021-08-27 00:42:48 +08:00)


## 创建模型输入

In [13]:
FEATURE_NAMES = [
    "fixed acidity",
    "volatile acidity",
    "citric acid",
    "residual sugar",
    "chlorides",
    "free sulfur dioxide",
    "total sulfur dioxide",
    "density",
    "pH",
    "sulphates",
    "alcohol",
]


def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        inputs[feature_name] = layers.Input(
            name=feature_name, shape=(1,), dtype=tf.float32
        )
    return inputs

time: 675 µs (started: 2021-08-27 00:43:28 +08:00)


## 实验一：标准神经网络
我们创建了一个标准的确定性神经网络模型作为基线。

In [14]:
def create_baseline_model():
    inputs = create_model_inputs()
    input_values = [value for _, value in sorted(inputs.items())]
    features = keras.layers.concatenate(input_values)
    features = layers.BatchNormalization()(features)

    # Create hidden layers with deterministic weights using the Dense layer.
    for units in hidden_units:
        features = layers.Dense(units, activation="sigmoid")(features)
    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

time: 1.21 ms (started: 2021-08-27 00:44:13 +08:00)


让我们将 wine 数据集拆分为训练集和测试集，分别有 85% 和 15% 的示例

In [15]:
dataset_size = 4898
batch_size = 256
train_size = int(dataset_size * 0.85)
train_dataset, test_dataset = get_train_and_test_splits(train_size, batch_size)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling wine_quality-train.tfrecord...:   0%|          | 0/4898 [00:00<?, ? examples/s]

Dataset wine_quality downloaded and prepared to /home/huangwei/tensorflow_datasets/wine_quality/white/1.0.0. Subsequent calls will reuse this data.
time: 1min 15s (started: 2021-08-27 00:44:48 +08:00)


现在让我们训练基线模型。 我们使用 MeanSquaredError 作为损失函数。

In [16]:
num_epochs = 100
mse_loss = keras.losses.MeanSquaredError()
baseline_model = create_baseline_model()
run_experiment(baseline_model, mse_loss, train_dataset, test_dataset)

Start training the model...
Epoch 1/100
17/17 [==============================] - 2s 64ms/step - loss: 41.6719 - root_mean_squared_error: 6.4554 - val_loss: 38.9705 - val_root_mean_squared_error: 6.2426
Epoch 2/100
17/17 [==============================] - 0s 21ms/step - loss: 39.1777 - root_mean_squared_error: 6.2592 - val_loss: 37.3527 - val_root_mean_squared_error: 6.1117
Epoch 3/100
17/17 [==============================] - 0s 20ms/step - loss: 37.0985 - root_mean_squared_error: 6.0909 - val_loss: 35.6854 - val_root_mean_squared_error: 5.9737
Epoch 4/100
17/17 [==============================] - 0s 19ms/step - loss: 35.1338 - root_mean_squared_error: 5.9274 - val_loss: 33.9785 - val_root_mean_squared_error: 5.8291
Epoch 5/100
17/17 [==============================] - 0s 20ms/step - loss: 33.2495 - root_mean_squared_error: 5.7662 - val_loss: 32.2820 - val_root_mean_squared_error: 5.6817
Epoch 6/100
17/17 [==============================] - 0s 19ms/step - loss: 31.4391 - root_mean_squared_

我们从测试集中抽取样本使用该模型来获得对它们的预测。 请注意，由于基线模型是确定性的，因此我们为每个测试示例得到一个单点估计预测，没有关于模型和预测的不确定性的信息。

In [17]:
sample = 10
examples, targets = list(test_dataset.unbatch().shuffle(batch_size * 10).batch(sample))[
    0
]

predicted = baseline_model(examples).numpy()
for idx in range(sample):
    print(f"Predicted: {round(float(predicted[idx][0]), 1)} - Actual: {targets[idx]}")

Predicted: 6.3 - Actual: 8.0
Predicted: 6.6 - Actual: 6.0
Predicted: 6.3 - Actual: 6.0
Predicted: 6.6 - Actual: 7.0
Predicted: 6.0 - Actual: 6.0
Predicted: 5.5 - Actual: 5.0
Predicted: 5.6 - Actual: 6.0
Predicted: 6.5 - Actual: 7.0
Predicted: 6.3 - Actual: 7.0
Predicted: 5.5 - Actual: 5.0
time: 166 ms (started: 2021-08-27 00:46:45 +08:00)
